In [1]:
import datetime


import ee
import pandas as pd

from IPython.display import Image
import openet.sims
import openet.core

ee.Initialize()

In [7]:
import pprint
import ee
import openet.core

ee.Initialize()

region = ee.FeatureCollection('projects/openet/api_demo_features').first().geometry()
# print(region.getInfo())

start_date = '2016-01-01'
end_date = '2016-03-31'
exclusive_end_date = ee.Date(end_date).advance(1, 'day').format('yyyy-MM-dd').getInfo()

scene_coll = ee.ImageCollection('projects/openet/geesebal/landsat/c02')\
  .filterDate(start_date, exclusive_end_date)\
  .filterBounds(region)

interp_bands = ['et']


def date_to_time_0utc(date):
    """Get the 0 UTC time_start for a date

    Parameters
    ----------
    date : ee.Date

    Returns
    -------
    ee.Number

    """
    return ee.Date.fromYMD(date.get('year'), date.get('month'),
                           date.get('day')).millis()
    # Extra operations are needed since update() does not set milliseconds to 0.
    # return date.update(hour=0, minute=0, second=0).millis()\
    #     .divide(1000).floor().multiply(1000)

    
# CGM - It might make more sense to have this inside the interpolation
def interpolate_prep(img):
    """Prep WRS2 scene images for interpolation

    "Unscale" the images using the "scale_factor" property and
        convert to double.
    Add a mask and time band to each image in the scene_coll since
        interpolator is assuming time and mask bands exist.
    The interpolation could be modified to get the mask from the
        time band instead of setting it here.
    The time image must be the 0 UTC time

    """
    mask_img = img.select([interp_bands[0]], ['mask'])\
        .multiply(0).add(1).updateMask(1).uint8()
    time_img = img.select([interp_bands[0]], ['time'])\
        .double().multiply(0)\
        .add(date_to_time_0utc(ee.Date(img.get('system:time_start'))))

    # TODO: This should probably check if the scale_factor property
    # TODO:   exists and is not 1
    return img.select(interp_bands) \
        .double().multiply(ee.Number(img.get('scale_factor')))\
        .addBands([mask_img, time_img])\
        .set({'system:time_start': ee.Number(img.get('system:time_start'))})
    #     .set({'image_id': ee.String(img.get('system:index'))})

scene_coll = scene_coll.map(interpolate_prep)


model_args = {
    'et_reference_source': 'projects/earthengine-legacy/assets/projects/openet/reference_et/gridmet/daily',
    'et_reference_band': 'eto',
    'et_reference_factor': 1.0,
    'et_reference_resample': 'nearest',
}
interp_args = {
    'interp_days': 32,
    'interp_method': 'LINEAR',
    'interp_source': 'projects/earthengine-legacy/assets/projects/openet/reference_et/gridmet/daily',
    'interp_band': 'eto'
}

pprint.pprint([[row[0], row[-1]] for row in list(scene_coll.getRegion(region.centroid(), 10).getInfo())])

print('Interpolating')
interp_coll = openet.core.interpolate.from_scene_et_actual(
    scene_coll=scene_coll, start_date=start_date, end_date=exclusive_end_date, variables=['et'], 
    interp_args=interp_args, model_args=model_args, t_interval='daily', use_joins=False)

print('{} images'.format(interp_coll.size().getInfo()))
pprint.pprint([[row[0], row[-1]] for row in list(interp_coll.getRegion(region.centroid(), 10).getInfo())])


[['id', 'time'],
 ['lc08_043034_20160112', None],
 ['lc08_043034_20160128', None],
 ['lc08_043034_20160213', 1455321600000],
 ['lc08_043034_20160229', 1456704000000],
 ['lc08_043034_20160316', 1458086400000],
 ['lc08_044034_20160204', None],
 ['lc08_044034_20160220', None],
 ['lc08_044034_20160323', None],
 ['le07_043034_20160205', 1454630400000],
 ['le07_043034_20160221', None],
 ['le07_043034_20160308', None],
 ['le07_043034_20160324', None],
 ['le07_044034_20160127', None],
 ['le07_044034_20160212', None],
 ['le07_044034_20160315', None],
 ['le07_044034_20160331', None]]
Interpolating
91 images
[['id', 'et'],
 ['20160101', None],
 ['20160102', None],
 ['20160103', None],
 ['20160104', 0.3941885232925415],
 ['20160105', 0.3220471739768982],
 ['20160106', 0.3112660050392151],
 ['20160107', 0.34166550636291504],
 ['20160108', 0.3775670826435089],
 ['20160109', 0.48854193091392517],
 ['20160110', 0.41983333230018616],
 ['20160111', 0.44882431626319885],
 ['20160112', 0.4695809781551361]